# Interferometry Image Reconstruction

This notebook provides a minimal image reconstruction algorithm for interferometry.

Let $I(\alpha, \beta)$ be an image on the sky.
Interferometry allows astronomer to sample the Fourier transform of the image $\hat{I}(u, v) = \int\int I(\alpha,\beta) \exp[-2\pi i(u\alpha + v\beta)] d\alpha d\beta$ along the $uv$-tracks $[u_{ab}(t), v_{ab}(t)]$ for telescopes $a$ and $b$ at time $t$.

Given a small number of telescope locations, we have a sparse set of $uv$ tracks, which sample the Fourier transform of the image.
We call these sampled Fourier coefficients the visibility $V_{abj} = \hat{I}(u_{ab}(t_n), v_{ab}(t_n)$ for all $a$, $b$, and $n$.

The problem of interferometry image reconstruction is to receover the original sky image $I(\alpha, \beta)$ to the best we can, based on the visibility $\{V_{abj}\}$.